# What is PyGiNaC, anyway?
### [Vladimir V Kisil](http://www1.maths.leeds.ac.uk/~kisilv/)

<a name="ToC"></a>
**Table of Contents**

+ [Introduction](#introduction)
+ [A glimpse into usage](#glimpse)
+ [Similarity and differences between GiNaC and PyGiNaC](#similarity)
+ [Downloads](#downloads)
+ [Compiling from source](#compilation)
  - [Prerequisites](#prerequisites)
  - [Compilation and test](#andtest)
  - [Installation](#installation)
  - [Building Debian package](#building)
+ [Mailing list](#mailing)
+ [TODO](#TODO)
+ [History and contributors](#history)
+ [Appendix: from Ginsh to PyGiNaC](#appendix)
  - [Arbitrary precision arithmetic](#arithmetic)
  - [Float point evaluation and constants](#rounding)
  - [Mathematical functions](#functions)
  - [Linear algebra](#linear)
  - [Polynomials and rational functions](#polynomials)
  - [Calculus](#calculus)
  - [Numeric methods](#numeric)
  - [Parser and integration with other packages](#parser)
+ [References](#references)

<a name="introduction"></a>
## Introduction

PyGiNaC is a Python package that provides an interface to the C++
library [GiNaC](http://www.ginac.de/), which is an open framework for
symbolic computation within C++. PyGiNaC is implemented with the help of
the [Boost.Python](http://www.boost.org/libs/python/doc/index.html)
library. At the moment, the package is more or less in an development
state, i.e.

+ the GiNaC classes are only partially exposed (yet most common methods are covered);

+ parts of regression test suite are unconverted; and 

+ no actual documentation exists (but who is reading the full documentation nowadays if you can quickly ask at StackOverflow?) 

In short: many things are already usable and further improvements 
are possible.

[Back to ToC](#ToC)

<a name="glimpse"></a>
## A glimpse into usage

Despite of being not-so-complete, PyGiNaC can do some fancy stuff for
you. For example, solving a linear system of equations in the Python
intepreter can be as simple as

In [1]:
from ginac import *
x = symbol('x')
y = symbol('y')
lsolve([3*x + 5*y == 2, 5*x+y == -3], [x,y])

{<cginac.symbol at 0x7f3eac27e7b0>: <cginac.numeric at 0x7f3eac27db20>,
 <cginac.symbol at 0x7f3eac27e710>: <cginac.numeric at 0x7f3eac27db90>}

The result is returned as a dictionary. To see it in a meaningful form we convert GiNac symbol and numeric objects to strings:

In [2]:
soln = lsolve([3*x + 5*y == 2, 5*x+y == -3], [x,y])
[f'{str(x)} : {str(soln[x])}' for x in soln]

['x : -17/22', 'y : 19/22']

Power series of functions are also handled:

In [3]:
x = symbol('x')
str(sin(x).series(x==0, 8))

'1*x+(-1/6)*x**3+1/120*x**5+(-1/5040)*x**7+Order(x**8)'

We also may enjoy from LaTeX pretty-printing:

In [2]:
from IPython.display import Latex
latex_on()
Latex(f'${sin(x).series(x==0, 8)}$')

<IPython.core.display.Latex object>

Here is a simple example of algebraic expansion:

In [5]:
x=realsymbol("x")
e=pow(x+1,2)
Latex(f'${e.expand()}$')

<IPython.core.display.Latex object>

A less obvious example of an algebraic expansion and simplification with exact arithmetic is a modified version of one of Ramanujan's identities (this example is ripped off from GiNaC's regression test suite):

In [6]:
e1 = pow(1 + pow(3, numeric(1,5)) - pow(3, numeric(2,5)),3)
e2 = (e1 - 10 + 5*pow(3, numeric(3,5)))
display(Latex(f'e2 is ${e2}$'))
f'e2 expands to {e2.expand()}'

<IPython.core.display.Latex object>

'e2 expands to 0'

Above `numeric(3,5)` is the fraction $\frac{3}{5}$. 

Further examples can be found below in the [Appendix](#appendix).

[Back to ToC](#ToC)

<a name="similarity"></a>
## Similarity and differences between GiNaC and PyGiNaC

Currently, (at least partially) exposed GiNaC classes are:

+ Basic (the GiNaC root class).
+ Constant.
+ Numeric.
+ Symbol, realsymbol, possymbol.
+ Expression, which are sums, products, non-commutative products, or powers of other expressions or basic objects.
+ Relational.
+ Matrix.
+ Function.
+ Indices.
+ Tensor.
+ Clifford unit.
+ Series.
+ Integral.
+ Lists are converted bidirectionally to Python3 lists.
+ Wildcard.
+ Symmetry.
+ Parser.
+ More technical: flags, exmap, expairs, expairseq, symtab.

Names of classes and methods in PyGiNaC are as close as possible to their prototypes in GiNaC. In most cases you can use [GiNaC Tutorial](https://ginac.de/tutorial/) as your user manual of PyGiNaC. Some inevitable differences come from the gap between C++ and Python, for example:

+ There are no semicolons at the end of statements and you need to use the proper Python identification in your scripts. 
+ Namespace syntax in C++ like  `subs_options::no_pattern` shall be replaced by Pythonish `subs_options.no_pattern`.
+ List initialisation in C++ like `lst l = {a, b, c}` shall be replaced by `l=[a, b, c]`.
+ You can use Python dictionaries in expression substitutions instead of list of relationals. For example, C++ statement
  ``` C++
  e.subs(lst{x==2, y==a+3});
  ```
  can be replaced by the Python line with list
  ``` Python
  e.subs([x==2, y==a+3])
  ```
  or with the Python dictionary:
  ``` Python
  e.subs({x : 2, y : a+3})
  ```
  See [an example](#dictsubexample) below.  

[Back to ToC](#ToC)

<a name="downloads"></a>
## (Non-)Downloads

You can try PyGiNaC without any local installation from two cloud services. Since Python wrapper for [MoebInv libraries](http://moebinv.sourceforge.net/) are built on top of PyGiNaC, the full access to PyGiNaC is provided in the following cloud Jupyter notebooks:

* [GitHub + Google CoLab](https://github.com/vvkisil/MoebInv-notebooks).

* [CodeOcen capsule](https://codeocean.com/capsule/7952650/tree).

If you want to have PyGiNaC installed locally you can use:

* [Debian binary packages](https://sourceforge.net/projects/moebinv/files/binary/debian/)

* [Source tarballs](https://sourceforge.net/projects/pyginac.moebinv.p/files/Releases/)

* [Git repository](https://sourceforge.net/p/moebinv/pyginac/code/ci/master/tree/)

If the pre-compiled Debian package is not working for your system you can compile the binary file as described in the next section.

[Back to ToC](#ToC)

<a name="compilation"></a>
## Compiling from source

<a name="prerequisites"></a>
### Prerequisites

To run PyGiNaC you need to have the following software installed:

+ [Python 3](http://www.python.org/) (tested for 3.6 or higher)

+ [The boost libraries](http://www.boost.org/) (tested for 1.65.0 or higher)

+ [GiNaC](http://www.ginac.de/) (tested for 1.7.2 or higher)

In addition to the above, to compile PyGiNaC the following are needed

+ [GNU make](https://www.gnu.org/s/make/manual/make.html)

+ [GNU g++](http://gcc.gnu.org/) (tested for 8 or higher).

Compiling PyGiNaC takes considerable memory (but not so high for modern computers), although generally not as much is needed to run it.

<a name="andtest"></a>
### Compilation and test
Once you have all the dependencies listed above installed, issue the command 
``` Shell
$ make
```
from the source directory. This will build the module in-place. The script `run` can be used to start an interactive Python session that will be able to use PyGiNaC by
``` Shell
$ ./run python3
```
or, to run a Python script
``` Shell
$ ./run python3 some_script_file.py
```
For example you can run the collection of self-test by
``` Shell
$ ./run python3 bin/checkall.py
```

That's it. Have fun. :)

<a name="installation"></a>
### Installation

To install PyGiNaC globally run 
``` Shell
$ make install
```
as the root from the source directory. Optional installation prefix, e.g. `/opt` can be specified as follows:
``` Shell
$ make install DESTDIR=/opt
```

<a name="building"></a>
### Building Debian package

If you are on Debian/Ubuntu system and with to re-build the binary file for some reasons you can do this with the standard command
``` Shell
$ debuild -us -uc
```
The relevant Debian packaging infrastructure need to be installed for this, of course.

[Back to ToC](#ToC)

<a name="mailing"></a>
## Mailing list

There is no currently a PyGiNaC related mailing list. Feel free to write an email to the curent maintainer at <kisilv@maths.leeds.ac.uk>.

[Back to ToC](#ToC)

<a name="TODO"></a>
## TODO

The following were identified by founding fathers as further targets:

* Wrap more of GiNaC classes and objects.

* Pythonize the GiNaC regression tests.

* Create a map_function system, probably based on Python function objects.

* Add extra member access functions for higher-level access of containerish types, like power.basis() and power.exponent(), for example.

* Make every function capable of taking more than one argument also take arguments in keyword form.

* Prepare some documention with examples written in Python instead of ginsh or C++.

Feel free to contribute to this or other worthy developments.

[Back to ToC](#ToC)

<a name="history"></a>
## History and contributors

The current implementation of PyGiNaC has, up to our knowledge, two predecessors: [a version](http://cens.ioc.ee/projects/pyginac/) written by Pearu Peterson many years ago, and [another one](http://ondrej.certik.cz/pyginac.php) by Ondrej Certik.

The present version of PyGiNaC is originally written by Jonathan Brandmeyer and later co-authored by Matti Peltomäki. Patches have been submitted by Ondrej Certik. Here is the [historic site](http://pyginac.sourceforge.net/).

Vladimir V. Kisil is the current maintainer of the [PyGiNaC code](https://sourceforge.net/projects/pyginac.moebinv.p/) as a subproject of [MoebInv](http://moebinv.sourceforge.net/) -- C++ libraries for symbolic, numeric and graphical manipulations in non-Euclidean geometry.

[Back to ToC](#ToC)

<a name="appendix"></a>
## Appendix: from Ginsh to PyGiNaC

This section is taken from the GiNaC Tutorial ["2.2 What it can do for you"](https://ginac.de/tutorial/#What-it-can-do-for-you) and shows how you can migrate your interactive usage from `Ginsh/C++` to `PyGiNaC`.
After invoking Python3/IPython/Jupyter shell one can test and experiment with PyGiNaC's features much like in other Computer Algebra Systems and mix it with arbitrary Python3 programming constructs like loops or conditionals. In IPython/Jupyter you can additionally benefit from extra features and magics, e.g. pretty-printed mathematics output. 

<a name="arithmetic"></a>
### Arbitrary precision arithmetic

(Py)GiNaC can manipulate arbitrary precision integers in a very fast way. Rational numbers are automatically converted to fractions of coprime integers:

In [7]:
x=pow(3,150)
str(x)

'369988485035126972924700782451696644186473100389722973815184405301748249'

Note that a statement `x=3**150` would produce a Python long integer instance. We need to use the dedicated function `pow()` that the PyGiNaC will take the precedence from the interpreter. 

A slightly different techniques to create a (Py)GiNaC numeric is:

In [8]:
y=numeric(3)**149
str(y)

'123329495011708990974900260817232214728824366796574324605061468433916083'

The next two results are exact numbers:

In [9]:
str(x/y)

'3'

To pretty-print the next output we use LaTeX facilities:

In [10]:
Latex(f'${y/x}$')

<IPython.core.display.Latex object>

These may be compared to the ordinary Python arithmetic:

In [11]:
(3**149)/(3**150)

0.3333333333333333

Exact numbers are always retained as exact numbers and only evaluated as floating point numbers if requested. For instance, with numeric radicals is dealt pretty much as with symbols. Products of sums of them can be expanded:

In [12]:
a=symbol("a")
Latex(f'${expand((1+a**numeric(1,5)-a**numeric(2,5))**3)}$')

<IPython.core.display.Latex object>

In [13]:
Latex(f'${expand((1+3**numeric(1,5)-3**numeric(2,5))**3)}$')

<IPython.core.display.Latex object>

[Back to ToC](#ToC)

<a name="rounding"></a>
### Float point evaluation and constants

A float point evaluation can be requested at any time:

In [14]:
Latex(f'${evalf((1+3**numeric(1,5)-3**numeric(2,5))**3)}$')

<IPython.core.display.Latex object>

The function `evalf()` that was used above converts any number in (Py)GiNaC's expressions into floating point numbers. This can be done to arbitrary predefined accuracy:

In [15]:
Latex(f'${evalf(numeric(1,7))}$')

<IPython.core.display.Latex object>

Now we change the required number of evaluated digits:

In [16]:
set_digits(150)
Latex(f'${evalf(numeric(1,7))}$')

<IPython.core.display.Latex object>

Exact numbers other than rationals that can be manipulated in (Py)GiNaC include predefined constants like Archimedes' $\pi$, called `Pi` in (Py)GiNaC. They can both be used in symbolic manipulations (as an exact number) as well as in numeric expressions (as an inexact number):

In [17]:
set_digits(15)
x=symbol("x")
a=Pi**2+x
Latex(f'${a}$')

<IPython.core.display.Latex object>

In [18]:
Latex(f'${evalf(a)}$')

<IPython.core.display.Latex object>

In [19]:
Latex(f'${evalf(a.subs(x==2))}$')

<IPython.core.display.Latex object>

<a name="dictsubexample"></a>
The same result is achieved through substitution with Python dictionary:

In [20]:
Latex(f'${evalf(a.subs({x : 2}))}$')

<IPython.core.display.Latex object>

(Py)GiNaC does not provide the Euler constant $e$, because it is primary needed as a base of the exponent function, see the next subsection.

[Back to ToC](#ToC)

<a name="functions"></a>
### Mathematical functions
(Py)GiNaC is aware of main mathematical functions and can manipulate them either in the exact or an approximate manner. 
For example, for the above mentioned exponential function $\exp(x)=e^x$,  (Py)GiNaC  knows the Euler identity:

In [21]:
X=exp(I*Pi)+1
str(X)

'0'

Of course, the Euler constant $e$ can be created as `exp(1)`:

In [22]:
E=exp(1)
Latex(f'${E}$')

<IPython.core.display.Latex object>

It has the expected value:

In [23]:
str(evalf(E))

'2.7182818284590452354'

But (Py)GiNaC is reluctant to make a reduction based on the Euler identity for *power* function with complex exponent: 

In [24]:
Latex(f'${eval(pow(E,I*Pi)+1)}$')

<IPython.core.display.Latex object>

To see the reason think about the identity $e^0 = e^{2\pi i}$ which would imply $(e^0)^i =( e^{2\pi i})^i$.

Built-in functions can be evaluated to exact numbers if this is possible.

In [25]:
Latex(f'${cos(42*Pi)}$')

<IPython.core.display.Latex object>

In [26]:
Latex(f'${cos(42*Pi).eval()}$')

<IPython.core.display.Latex object>

Conversions that can be safely performed are done immediately; conversions that are not generally valid are not done:

In [27]:
Latex(f'${cos(acos(x))}$')

<IPython.core.display.Latex object>

In [28]:
Latex(f'${cos(acos(x)).eval()}$')

<IPython.core.display.Latex object>

However we have:

In [29]:
Latex(f'${acos(cos(x)).eval()}$')

<IPython.core.display.Latex object>

Note that converting the last input to $x$ would allow one to conclude that $42 \pi$ is equal to 0.

[Back to ToC](#ToC)

<a name="linear"></a>
### Linear Algebra

Linear equation systems can be solved along with basic linear algebra manipulations over symbolic expressions. In (Py)GiNaC offers a `matrix class`. We start from a single equation:

In [30]:
a=symbol("a")
x=symbol("x")
y=symbol("y")
z=symbol("z")
soln=lsolve([a+x*y==z], [x])
Latex(f'${soln[x]}$')

<IPython.core.display.Latex object>

A pair of linear equations with two variables:

In [31]:
soln=lsolve([3*x+5*y == 7, -2*x+10*y == -5], [x, y])
for t in soln:
    display(Latex(f'${t}$ : ${soln[t]}$'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

A matrix can be created from a list of lists of row elements:

In [32]:
M = matrix([ [1, 3], [-3, 2] ])
Latex(f'${M.determinant()}$')

<IPython.core.display.Latex object>

The characteristic polynomial (note that `lambda` is a reserved keyword in Python3):

In [33]:
lam=symbol("lambda")
Latex(f'${M.charpoly(lam)}$')

<IPython.core.display.Latex object>

Matrix operations can be called in a usual way:

In [34]:
A = matrix([ [1, 1], [2, -1] ])
Latex(f'${A+2*M}$')

<IPython.core.display.Latex object>

However their evaluation is postponed until an explicit request by the dedicated function `evalm()` for matrix evaluation:

In [35]:
Latex(f'${evalm(A+2*M)}$')

<IPython.core.display.Latex object>

Matrix arithmetic is also performed in the exact manner:

In [36]:
a=symbol("a")
b=symbol("b")
B = matrix([ [0, 0, a], [b, 1, -b], [-1/a, 0, 0] ])
Latex(f'${evalm(B**(2**12345))}$')

<IPython.core.display.Latex object>

[Back to ToC](#ToC)

<a name="polynomials"></a>
### Polynomials and rational functions

Multivariate polynomials and rational functions may be expanded, collected and normalized (i.e. converted to a ratio of two coprime polynomials):

In [37]:
a = x**4 + 2*x**2*y**2 + 4*x**3*y + 12*x*y**3 - 3*y**4
Latex(f'${a}$')

<IPython.core.display.Latex object>

In [38]:
b = x**2 + 4*x*y - y**2
Latex(f'${b}$')

<IPython.core.display.Latex object>

In [39]:
Latex(f'${expand(a*b)}$')

<IPython.core.display.Latex object>

In [40]:
Latex(f'${collect(a+b,x)}$')

<IPython.core.display.Latex object>

In [41]:
Latex(f'${collect(a+b,y)}$')

<IPython.core.display.Latex object>

In [42]:
Latex(f'${normal(a/b)}$')

<IPython.core.display.Latex object>

[Back to ToC](#ToC)

<a name="calculus"></a>
### Calculus
You can differentiate any expression as follows:

In [43]:
Latex(f'${diff(tan(x),x)}$')

<IPython.core.display.Latex object>

Any expression can be expanded as Taylor or Laurent series in a very natural syntax (the second argument of series is a relation defining the evaluation point, the third specifies the order):

In [44]:
Latex(f'${series(sin(x),x==0,4)}$')

<IPython.core.display.Latex object>

In [45]:
Latex(f'${series(1/tan(x),x==0,4)}$')

<IPython.core.display.Latex object>

Or a bit more involved 

In [46]:
Latex(f'${series(tgamma(x),x==0,3)}$')

<IPython.core.display.Latex object>

In necessary a floating point evaluation can be called as well:

In [47]:
Latex(f'${series(tgamma(x),x==0,3).evalf()}$')

<IPython.core.display.Latex object>

(If the last output is identical to the previous one, this shall be due to a GiNaC bug up to the version $\leq$ 1.7.7).

In [48]:
Latex(f'${series(tgamma(2*sin(x)-2),x==Pi/2,6)}$')

<IPython.core.display.Latex object>

What about integration? Let us try:

In [3]:
t=realsymbol("t")
Int=integral(x, 0, t, x*x+sin(x))
display(Latex(f'${Int}$'))
Latex(f'${Int.eval_integ()}$')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

That is: (Py)GiNaC is not very useful at symbolically evaluating integrals, it can do it for polynomials only. However, (Py)GiNaC is aware of the Fundamental Theorem of Calculus:

In [60]:
Latex(f'${Int.diff(t)}$')

<IPython.core.display.Latex object>

Furthermore,  (Py)GiNaC is not so bad at the numeric evaluation of definite integrals, see the next subsection.

[Back to ToC](#ToC)

<a name="numeric"></a>
### Numeric methods

Reconsider the above integral and ask its numerical evaluation: 

In [66]:
Latex(f'${integral(x, 0, 1, x*x+sin(x)).evalf()}$')

<IPython.core.display.Latex object>

See GiNaC Tutorial for fine-tuning of numerical integration.

Often, functions don't have roots in closed form. Nevertheless, it's quite easy to compute a solution numerically, to arbitrary precision:

In [50]:
set_digits(50)
str(fsolve(cos(x)==x,x,0,2))

'0.7390851332151606416553120876738734040134117589007574649658'

In [51]:
f=exp(sin(x))-x
X=fsolve(f,x,-10,10)
str(X)

'2.2191071489137460325957851882042901681753665565320678854155'

In [52]:
str(subs(f,x==X))

'-6.372367644529809108115521591070847222364418220770475144296E-58'

Notice how the final result above differs slightly from zero by about $6\cdot 10^{-58}$. This is because with 50 decimal digits precision the root cannot be represented more accurately than X. Such inaccuracies are to be expected when computing with finite floating point values.

If you ever wanted to convert units in C or C++ and found this is cumbersome, here is the solution. Symbolic types can always be used as tags for different types of objects. Converting from wrong units to the metric system is now easy:

In [53]:
set_digits(10)
m=symbol("m")
kg=symbol("kg")
inch=.0254*m
lb=.45359237*kg
Latex(f'${(200*lb/inch**2).evalf()}$')

<IPython.core.display.Latex object>

[Back to ToC](#ToC)

<a name="parser"></a>
## Parser and integration with other packages

(Py)GiNaC has some attractive features, but is not a one-for-all package. You may want to integrate it with other available packages. For example, (Py)GiNaC can parse mathematical expressions from strings produced by a user or software. This can be as simple as this:

In [7]:
reader=parser()
e=reader("Pi^2+sin(2*t)")
Latex(f'${e}$')

<IPython.core.display.Latex object>

Note C++-style notation for powers, the (Py)GiNaC expects these rather than pythonish expressions. The output looks nice but you may be disappointed by the next line:

In [9]:
str(e.diff(t))

'0'

In fact, our `reader` was not aware of the previously defined symbol $t$ and created a new symbol still represented by  the same letter. To avoid such a confusion the `reader` need to be aware of desirable substitutions from strings to existing symbols (or even expressions):

In [15]:
reader=parser({"t" : t})
e=reader("Pi^2+sin(2*t)")
display(Latex(f'e=${e}$'))
Latex(r'$\frac{de}{dt}='+f'{e.diff(t)}$')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

We can disable parsing of unknown symbols (for example, you want treat an unexpected string in the input as an error): 

In [19]:
reader.set_strict(True)
str(reader("1+2*z"))

'1+2 z'

and enable it back:

In [22]:
reader.set_strict(False)
str(reader("1+2*z"))

'1+2 z'

You can obtain all parser-generated symbols with 'get_syms()' method:

In [20]:
D=reader.get_syms()
[f'"{x}" : {str(D[x])}' for x in D]

['"t" : t', '"z" : z']

The dictionary of known symbols can be updated as needed:

In [24]:
D.update({"w" : log(y)})
reader.set_syms(D)
Latex(f'${reader("t+s+w")}$')

<IPython.core.display.Latex object>

Parsers provide a convenient communication tool with other mathematical packages or user inputs. Fine-tuning can be achieved by additional string manipulations. 

Hopefully, the above examples are sufficient to give an idea how PyGiNaC can be used. Further advice can be found in GiNaC tutorial, PyGINaC test suit and MoebInv notebooks, see references below. 

[Back to ToC](#ToC)

<a name="references"></a>
## References

1. [PyGiNaC--maintained version](https://sourceforge.net/projects/pyginac.moebinv.p/)
2. <a name="tutorial"></a>[GiNaC Tutorial](https://ginac.de/tutorial/).
3. <a name="notebooks"></a>[MoebInv Notebooks](https://github.com/vvkisil/MoebInv-notebooks/)